In [1]:
import os

In [2]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\brain\\TUMOR-DETECTION-MLOPS-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\User\\Desktop\\PROJECTS\\brain\\TUMOR-DETECTION-MLOPS-'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool



@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf
import pandas as pd

In [35]:
import pandas as pd
from pathlib import Path

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        # training_data = pd.read_csv(os.path.join(self.config.data_ingestion.unzip_dir, "data_mask.csv"))
        training_data = pd.read_csv('artifacts/data_ingestion/Brain_MRI/data_mask.csv')
        create_directories([
                Path(training.root_dir)
            ])

        # csv_path = "artifacts/data_ingestion/Brain_MRI/data_mask.csv"
        # training_data = pd.read_csv(csv_path)

        # training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        # create_directories([
        #     Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=training_data,
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
        )

        return training_config


In [36]:
import time

In [43]:
class PrepareCallback:
    def __init__(self, config):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        monitor = str(Path('val_loss'))
        checkpoint_filepath = str(self.config.checkpoint_model_filepath)  # Convert WindowsPath object to string

        early_stopping = tf.keras.callbacks.EarlyStopping(monitor=monitor, mode='min', verbose=1, patience=20)
        checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_best_only=True)

        return early_stopping, checkpointer

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,  # Invoke the _create_tb_callbacks method
            self._create_ckpt_callbacks  # Invoke the _create_ckpt_callbacks method
        ]


In [44]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import pandas as pd

In [47]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    def train_valid_generator(self):
        self.config.training_data['mask'] = self.config.training_data['mask'].apply(lambda x: str(x))
        
        # Specify the full path to the 'Brain_MRI' directory
        directory = os.path.join('artifacts/data_ingestion/Brain_MRI')
        
        datagenerator_kwargs = dict(
            rescale=1./255.,
            validation_split=0.15
        )

        dataflow_kwargs = dict(
            x_col='image_path',
            y_col='mask',
            batch_size=self.config.params_batch_size,
            class_mode="categorical",
            target_size=(256, 256)
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_dataframe(
            dataframe=self.config.training_data,
            directory=directory,
            subset="validation",
            shuffle=True,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_dataframe(
            dataframe=self.config.training_data,
            directory=directory,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.n // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.n // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [48]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2023-07-12 02:01:00,184: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-07-12 02:01:00,192: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-12 02:01:00,196: INFO: common: created directory at: artifacts]
[2023-07-12 02:01:00,199: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-07-12 02:01:00,214: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-07-12 02:01:00,253: INFO: common: created directory at: artifacts\training]


Found 589 validated image filenames belonging to 2 classes.
Found 3340 validated image filenames belonging to 2 classes.
208/208 [==============================] - ETA: 0s - loss: 0.7329 - accuracy: 0.5626

c:\Users\User\anaconda3\envs\brain\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


208/208 [==============================] - 1132s 5s/step - loss: 0.7329 - accuracy: 0.5626 - val_loss: 0.6191 - val_accuracy: 0.9931
